In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import cv2

def process_dataset(dataset_path):
    """Process all images in dataset and save homographies"""
    # Get all images in dataset
    images = sorted(glob.glob(f'{dataset_path}/*/*/*.jpg'))
    

    
    for image_path in images:
        # Get corresponding annotation path
        if 'soccer_worldcup_2014' in image_path:
            gt_homo = np.loadtxt(image_path.replace('.jpg', '.homographyMatrix'))
        else:
            anno_path = image_path.replace("Dataset", "Annotations")
            # if not os.path.exists(anno_path):
            #     print(f"Skipping {image_path} - no annotation found")
            #     continue
                
            if 'IMG' in anno_path:
                gt_homo = np.load(anno_path.replace('.jpg', '_homography.npy'))
            else:
                gt_homo = np.load(anno_path.replace('.jpg', '.npy'))

        # Create output directory
        out_path = image_path.replace("Dataset", "processed_homographies")
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        
        # Save homography
        out_file = out_path.replace('.jpg', '.txt')
        np.savetxt(out_file, gt_homo)
        print(f"Processed {image_path}")



In [ ]:
import os
import glob
import numpy as np
import cv2
import sys
sys.path.append('src')
import utils

# Load dense template
template_dense = 1 - utils.gen_template_dense_features()

def process_dataset(dataset_path):
    """Process all images in dataset and save homographies"""
    # Get all images in dataset
    images = sorted(glob.glob(f'{dataset_path}/*/*/*.jpg'))
    
    for image_path in images:
        # Get corresponding annotation path
        if 'soccer_worldcup_2014' in image_path:
            gt_homo = np.loadtxt(image_path.replace('.jpg', '.homographyMatrix'))
        else:
            anno_path = image_path.replace("Dataset", "Annotations")
            # if not os.path.exists(anno_path):
            #     print(f"Skipping {image_path} - no annotation found")
            #     continue
                
            if 'IMG' in anno_path:
                gt_homo = np.load(anno_path.replace('.jpg', '_homography.npy'))
            else:
                gt_homo = np.load(anno_path.replace('.jpg', '.npy'))

        # Read image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (1280, 720))
        
        # Apply homography to dense template
        S = np.eye(3)
        S[0, 0] = image.shape[1] / 1280
        S[1, 1] = image.shape[0] / 720
        inv_homo = S @ np.linalg.inv(gt_homo)
        
        warped_dense = cv2.warpPerspective(template_dense, inv_homo,
                                         (image.shape[1], image.shape[0]),
                                         cv2.INTER_LINEAR,
                                         borderMode=cv2.BORDER_CONSTANT, 
                                         borderValue=(0)) * 255
        
        # Process dense visualization
        warped_dense[warped_dense < 240] = 0
        warped_dense = cv2.cvtColor(warped_dense, cv2.COLOR_GRAY2BGR).astype(np.uint8)
        warped_dense[:, :, [0, 2]] = 0  # Keep only green channel
        
        # Blend with original image
        dense_alpha = 0.3
        visualization = cv2.addWeighted(image, 1 - dense_alpha, warped_dense, dense_alpha, 0)

        # Create output directories
        homo_out_path = image_path.replace("Dataset", "processed_homographies")
        vis_out_path = image_path.replace("Dataset", "processed_visualizations") 
        os.makedirs(os.path.dirname(homo_out_path), exist_ok=True)
        os.makedirs(os.path.dirname(vis_out_path), exist_ok=True)
        
        # Save homography and visualization
        homo_out_file = homo_out_path.replace('.jpg', '.txt')
        vis_out_file = vis_out_path.replace('.jpg', '_dense.jpg')
        np.savetxt(homo_out_file, gt_homo)
        cv2.imwrite(vis_out_file, visualization)
        
        print(f"Processed {image_path}")

In [ ]:
import os
import glob
import numpy as np
import cv2
import sys
sys.path.append('src')
import utils

# Load template grid points instead of dense template
template_grid = utils.gen_template_grid(uniform=False, increase=False)

def process_dataset(dataset_path):
    """Process all images in dataset and save homographies"""
    # Get all images in dataset
    images = sorted(glob.glob(f'{dataset_path}/*/*/*.jpg'))
    
    for image_path in images:
        # Get corresponding annotation path
        if 'soccer_worldcup_2014' in image_path:
            gt_homo = np.loadtxt(image_path.replace('.jpg', '.homographyMatrix'))
        else:
            anno_path = image_path.replace("Dataset", "Annotations")
            # if not os.path.exists(anno_path):
            #     print(f"Skipping {image_path} - no annotation found")
            #     continue
                
            if 'IMG' in anno_path:
                gt_homo = np.load(anno_path.replace('.jpg', '_homography.npy'))
            else:
                gt_homo = np.load(anno_path.replace('.jpg', '.npy'))

        # Read image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (1280, 720))
        
        # Apply homography to template grid points
        S = np.eye(3)
        S[0, 0] = image.shape[1] / 1280
        S[1, 1] = image.shape[0] / 720
        inv_homo = S @ np.linalg.inv(gt_homo)
        
        # Transform grid points
        warped_points = cv2.perspectiveTransform(template_grid[:, :2].reshape(-1, 1, 2), inv_homo).squeeze()
        
        # Create visualization
        visualization = image.copy()
        
        # Draw lines connecting the grid points
        # Vertical lines
        for x in np.unique(template_grid[:, 0]):
            points = warped_points[template_grid[:, 0] == x]
            for i in range(len(points) - 1):
                pt1 = tuple(map(int, points[i]))
                pt2 = tuple(map(int, points[i + 1]))
                cv2.line(visualization, pt1, pt2, (0, 255, 0), 2)
                
        # Horizontal lines
        for y in np.unique(template_grid[:, 1]):
            points = warped_points[template_grid[:, 1] == y]
            for i in range(len(points) - 1):
                pt1 = tuple(map(int, points[i]))
                pt2 = tuple(map(int, points[i + 1]))
                cv2.line(visualization, pt1, pt2, (0, 255, 0), 2)

        # Create output directories
        homo_out_path = image_path.replace("Dataset", "processed_homographies")
        vis_out_path = image_path.replace("Dataset", "processed_visualizations") 
        os.makedirs(os.path.dirname(homo_out_path), exist_ok=True)
        os.makedirs(os.path.dirname(vis_out_path), exist_ok=True)
        
        # Save homography and visualization
        # Parse the image filename and extract the number
        filename = os.path.basename(image_path)
        file_number = re.search(r'\d+', filename)
        if file_number:
            file_number = file_number.group().zfill(4)  # Add leading zeros to make it 4 digits
        else:
            file_number = "0000"  # Default if no number is found

        # Update output filenames with the formatted number
        homo_out_file = homo_out_path.replace('.jpg', f'_{file_number}.txt')
        vis_out_file = os.path.join(os.path.dirname(vis_out_path), f'{file_number}_lines.jpg')
        np.savetxt(homo_out_file, gt_homo)
        cv2.imwrite(vis_out_file, visualization)
        
        print(f"Processed {image_path}")

In [ ]:
dataset_path = "/home/ptf/Apps/KeypointAnnotator/dataset/consolidated/Dataset"
process_dataset(dataset_path)

In [ ]:
import os
import glob
import subprocess
from pathlib import Path
import re

def natural_sort_key(s):
    """
    Sort strings containing numbers in natural order.
    Example: ['1.jpg', '2.jpg', '10.jpg'] instead of ['1.jpg', '10.jpg', '2.jpg']
    """
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', str(s))]


def create_videos_from_visualizations(vis_root_path, output_dir, fps=30):
    """
    Create videos from visualization images in subdirectories
    
    Args:
        vis_root_path: Root path containing visualization subdirectories
        output_dir: Directory to save output videos
        fps: Frames per second for output videos
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print("ololo")
    # Get all subdirectories containing visualizations
    subdirs = [d for d in Path(vis_root_path).glob('*/*') if d.is_dir()]
    print(subdirs)
    for subdir in subdirs:
        print(subdir)
        # Get all line visualization images in current subdir
        images = sorted(glob.glob(str(subdir / '*_lines.jpg')), key=natural_sort_key)

        # Check if there are any images        
        if not images:
            print(f"No visualization images found in {subdir}")
            continue
            
        # Create output video filename
        video_name = f"{subdir.parent.name}_{subdir.name}.mp4"
        video_path = os.path.join(output_dir, video_name)
        
        # FFmpeg command to create video
        ffmpeg_cmd = [
            'ffmpeg',
            '-y',  # Overwrite output files
            '-framerate', str(fps),
            '-i', f"{subdir}/%*_lines.jpg",  # Use natural sorting by specifying a pattern
            # '-c:v', 'libx264',
            # '-pix_fmt', 'yuv420p',
            # '-crf', '23',
            video_path
        ]
        
        try:
            print(f"Creating video for {subdir}")
            subprocess.run(ffmpeg_cmd, check=True, capture_output=True)
            print(f"Successfully created {video_path}")
        except subprocess.CalledProcessError as e:
            print(f"Error creating video for {subdir}")
            print(f"FFmpeg error: {e.stderr.decode()}")

vis_root = "/home/ptf/Apps/KeypointAnnotator/dataset/consolidated/processed_visualizations"
output_dir = "/home/ptf/Apps/KeypointAnnotator/dataset/consolidated/videos"

create_videos_from_visualizations(vis_root, output_dir)

In [ ]:
fi